# Intro to geomeppy_tutorial
---
##### As mentioned in eppy tutorial that eppy is a scripting language to Energyplus. Eppy puts the idd and idf file into a python data structure, and uses the power of python to edit idf files. Addition to Eppy, GeomEppy is a scripting language to Energyplus, primarily focuses on IDF geometry (zones, surfaces, constructions, etc.)
##### This tutorial is intended to help you take the first step in using GeomEppy. The demo file is a quick introduction of using GeomEppy to build geometries for idf files:
* access an idf file with geomeppy
* build blocks for the idf file, with zoning specified
* make window with wwr
* import construction from other idf files, and apply construction to the objects
* modified the built geomety: scaling, rotating, etc. 
* visualize and save the built geometry as obj file
* run energy plus on geomeppy
------

### Install geomeppy:
As we already covered in the eppy/geomeppy installation tutorial, you need to install geomeppy by **pip install geomeppy** in your **conda** environment, if you have already done so, skip this step.

In [ ]:
pip install geomeppy

--------


### Install EnergyPlus
Next install **EnergyPlus** if you have not done so. 

**EnergyPlus version**: This tutorial will be running with EnergyPlus 9.1.0. But if you are using other versions, don't worry, geomeppy will also work for your EnergyPlus version. Just remember to change the idd path properly to your version, and keep using one version of EnergyPlus in one jupyter launcher. 

Next, get the idd filepath from the EnergyPlus installation folder. This tutorial is on Mac, so the idd filepath would be: '/Applications/EnergyPlus-9-1-0/Energy+.idd'. 
On windows the path would usually be: 'C:/EnergyPlusV9-1-0', and on Linux '/usr/local/EnergyPlus-9-1-0'.

### Run geomeppy:
Then,we are going to import idf file to **geomeppy**. 

Pick an **idf file** that you wish you access. This tutorial uses the 'Minimal.idf' file from the energy plus example folder, the file path is: '/Applications/EnergyPlus-9-1-0/ExampleFiles/Minimal.idf'. 

With these ready, in the python 3 environment, type: 

In [ ]:
from geomeppy import IDF
IDF.setiddname('/Applications/EnergyPlus-9-1-0/Energy+.idd')
idf = IDF('/Applications/EnergyPlus-9-1-0/ExampleFiles/Minimal.idf')

Then we can add a a weather file to our idf object. This tutorial uses 'USA_CO_Golden-NREL.724666_TMY3.epw'

In [ ]:
idf.epw = 'USA_CO_Golden-NREL.724666_TMY3.epw'

Now we could start adding a simple block to the idf object by giving it a name, specifying its four base coordinates,its height, and number  of floors. 

In [ ]:
idf.add_block(
    name='Test Block1',
    coordinates=[(10,0),(10,5),(0,5),(0,0)],
    height=5,
    num_stories=2,)

You could specified zoning by storey

In [ ]:
idf.add_block(
    name='Test Block1',
    coordinates=[(10,0),(10,5),(0,5),(0,0)],
    height=5,
    num_stories=2,
    zoning="by_storey")

The zoning could also be specified as core/perim, and the depth of the perimeter could be defined.

In [ ]:
idf.add_block(
    name='Test Block2',
    coordinates=[(10,5),(10,10),(0,10),(0,5)],
    height=9,
    zoning="core/perim",
    perim_depth=2.0)

Before setting the windows, we need to intersect all surfaces, so walls of blocks won't get duplication.

In [ ]:
idf.intersect_match()

Next we can set default constructions for the rest of idf surfaces.

In [ ]:
idf.set_default_constructions()

Let's view all the default construction in this idf

In [ ]:
idf.idfobjects['construction'.upper()]

By using **copy_construction** from geomeppy.extractor, we could import construction from another idf file. 
Here let's try import window constrctions from WindowTestsSimple.idf in the examplefile

In [ ]:
from geomeppy.extractor import copy_constructions
src_idf = IDF('/Applications/EnergyPlus-9-1-0/ExampleFiles/WindowTestsSimple.idf')
copy_constructions(source_idf=src_idf, target_idf=idf)

Now let's view all the construction again and check if the window constructions have been imported

In [ ]:
idf.idfobjects["CONSTRUCTION"]

Then, we could create windows by giving it window to wall ratio(wwr), and construction type

In [ ]:
idf.set_wwr(0.25,construction="DOUBLE PANE WITH SCREEN")

We could view all windows and theri constructions

In [ ]:
allwindow=idf.idfobjects['FenestrationSurface:Detailed'.upper()]
allwindow

We could use Construction_name to change all the windows' construction to Double Pane Window

In [ ]:
for window in idf.idfobjects['FenestrationSurface:Detailed'.upper()]:
    window.Construction_Name='DOUBLE PANE WINDOW'
    print (window)

Then, we could use the **view_model** function to view the blocks with the windows

In [ ]:
idf.view_model()


<img src="images/1.png" width='30%'>


The blocks could be rotated by a defined degree:

In [ ]:
idf.rotate(90)


<img src="images/2.png" width='35%'>


And scaled on selected axis, here we double the dimension of the blocks on 'zy' axis:

In [ ]:
idf.scale(2,axes='zy')

<img src="images/3.png" width='30%'>

A note here, **.scale** is a function applied to the idf file, if the same code is run twice, the geometries' dimension in the idf file would be doubled twice in 'zy' axis

same for other functions to the idf file

You may save the geometry you just build and modified to an obj file

In [ ]:
idf.to_obj('test.obj')

We can view the idf file and check the modification you made to the idf file. 

In [ ]:
idf.printidf()

Finally, we can run the idf file on geomeppy

In [ ]:
idf.run()

A successful run in windows system would look  like this:

<img src="images/windows.png" width='40%'>

A successful run in mac system would look  like this:

<img src="images/mac.png" width='50%'>

The idf file could be saved or saved as:

In [ ]:
idf.save('Filepath to save the file')

In [ ]:
idf.saveas('Filepath to set the new idf file name attribute to and save the file as')

----
## more geomeppy resources
The source documentation of geomeppy could be found: https://geomeppy.readthedocs.io/en/latest/